In [2]:
import jax
from jax import random
import jax.numpy as jnp
import flax
import flax.linen as nn
from craftax_classic.envs.craftax_symbolic_env import CraftaxClassicSymbolicEnv
from environment_base.wrappers import AutoResetEnvWrapper
from ml_collections import FrozenConfigDict

from jax_code.models import QNet, Discriminator
from jax_code.diayn import train

In [4]:
key1, key2, key3 = random.split(random.key(0), 3)
x = random.uniform(key1, (5000, 8))
skill = random.uniform(key2, (5000, 3))

model = QNet(action_size=4, hidden1_size=64, hidden2_size=64, dropout_rate=0.2)
params = model.init(key2, x, skill, train=False)
y = model.apply(params, x, skill, train=True, rngs={'dropout': key3})
y.shape

(5000, 4)

In [5]:
key1, key2, key3 = random.split(random.key(0), 3)
x = random.uniform(key1, (5000, 8))

model = Discriminator(skill_size=3, hidden1_size=64, hidden2_size=64, dropout_rate=0.2)
params = model.init(key2, x, train=False)
y = model.apply(params, x, train=True, rngs={'dropout': key3})
y.shape

(5000, 3)

In [3]:
rng = jax.random.PRNGKey(0)
rng, _rng = jax.random.split(rng)
rngs = jax.random.split(_rng, 3)

# Create environment
env = AutoResetEnvWrapper(CraftaxClassicSymbolicEnv())
env_params = env.default_params

# Get an initial state and observation
obs, state = env.reset(rngs[0], env_params)
print(obs.shape)

# Pick random action
action = env.action_space(env_params).sample(rngs[1])

# Step environment
obs, state, reward, done, info = env.step(rngs[2], state, action, env_params)

(1345,)


/nas/ucb/arvindrajaraman/anaconda3/envs/diayn/lib/python3.9/site-packages/jax/_src/numpy/array_methods.py:66: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy.astype(arr, dtype)


In [5]:
for i in range(5000):
    obs, state, reward, done, info = env.step(rngs[2], state, action, env_params)
    print(done)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals